In [ ]:
import torch
import numpy as np
from models.resnet import load_resnet_model
from models.swin3D import load_swin3d
from models.swin import load_swin

from utils.train import train_model
from utils.dataloader import create_dataloader, create_dataset
from utils.utils import sorted_file_paths

import torch.nn as nn
import torch.optim as optim

from matplotlib import pyplot as plt
from loss.balanced_MSE_loss import BMCLoss
from loss.weighted_MSE_loss import WeightedMSELoss
from loss.dense_weight import DenseWeight

# Setting up common variables for all experiments

In [ ]:
LR = 0.001
EPOCH = 30
GAMMA = 0.95
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_files = sorted_file_paths('./data/records_location/train/')
val_files = sorted_file_paths('./data/records_location/val/')

train_dataset = create_dataset(train_files) 
val_dataset = create_dataset(val_files)

# If you want to train 3D model please use this instead
# train_dataset = create_dataset(train_files, reshape_3d: true) 
# val_dataset = create_dataset(val_files, reshape_3d: true)

train_loader = create_dataloader(train_dataset, BATCH_SIZE)
val_loader = create_dataloader(val_dataset, BATCH_SIZE)

training_labels = train_dataset.all_labels.numpy()

# Experiment 0: L2 loss Swin 3d

In [ ]:
ex = 'experiments/ex_0'
loss =  nn.MSELoss()
model = load_swin3d("base")
optimizer = optim.Adam(model.parameters(), LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train_model(model, loss, optimizer, scheduler, train_loader, val_loader, device, ex,epochs=EPOCH)

# Experiment 1: L2 loss Swin 3d tiny

In [ ]:
ex = 'experiments/ex_1'
loss =  nn.MSELoss()
model = load_swin3d("tiny")
optimizer = optim.Adam(model.parameters(), LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train_model(model, loss, optimizer, scheduler, train_loader, val_loader, device, ex, epochs=EPOCH)

# Experiment 2: L2 loss Resnet 50

In [ ]:
ex = 'experiments/ex_2'
loss =  nn.MSELoss()
model = load_resnet_model('resnet50', num_classes=1)
optimizer = optim.Adam(model.parameters(), LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train_model(model, loss, optimizer, scheduler, train_loader, val_loader, device, ex, epochs=EPOCH)

# Experiment 3: L2 Loss Swin

In [ ]:
ex = 'experiments/ex_3'
loss =  nn.MSELoss()
model = load_swin()
optimizer = optim.Adam(model.parameters(), LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train_model(model, loss, optimizer, scheduler, train_loader, val_loader, device, ex, epochs=EPOCH)

# Experiment 4: L3 loss

In [ ]:
from loss.weighted_L3_loss import L3Loss

ex = 'experiments/ex_4'
loss =  L3Loss()
model = load_resnet_model()
optimizer = optim.Adam(model.parameters(), LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train_model(model, loss, optimizer, scheduler, train_loader, val_loader, device, ex, epochs=EPOCH)

# Experiment 5-8: Weighted L2, dense weight alpha = [0.25, 0.5, 0.75, 1.0]

In [ ]:
ex = 'experiments/ex_5'

dense_weight_model = DenseWeight(0.25)
dense_weight_model.fit(training_labels)
loss =  WeightedMSELoss(dense_weight_model)
model = load_resnet_model()
optimizer = optim.Adam(model.parameters(), LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train_model(model, loss, optimizer, scheduler, train_loader, val_loader, device, ex, epochs=EPOCH)

In [ ]:
ex = 'experiments/ex_6/'

dense_weight_model = DenseWeight(0.5)
dense_weight_model.fit(training_labels)
loss =  WeightedMSELoss(dense_weight_model)
model = load_resnet_model()
optimizer = optim.Adam(model.parameters(), LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train_model(model, loss, optimizer, scheduler, train_loader, val_loader, device, ex, epochs=EPOCH)

In [ ]:
ex = 'experiments/ex_7/'

dense_weight_model = DenseWeight(0.75)
dense_weight_model.fit(training_labels)
loss =  WeightedMSELoss(dense_weight_model)
model = load_resnet_model()
optimizer = optim.Adam(model.parameters(), LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train_model(model, loss, optimizer, scheduler, train_loader, val_loader, device, ex, epochs=EPOCH)

In [ ]:
ex = 'experiments/ex_8/'

dense_weight_model = DenseWeight(1)
dense_weight_model.fit(training_labels)
loss =  WeightedMSELoss(dense_weight_model)
model = load_resnet_model()
optimizer = optim.Adam(model.parameters(), LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train_model(model, loss, optimizer, scheduler, train_loader, val_loader, device, ex, epochs=EPOCH)

# Experiment 9-12: Weighted Sampling with replacement, dense weight alpha = [0.25, 0.5, 0.75, 1.0]

In [ ]:
ex = 'experiments/ex_9/'

dense_weight_model = DenseWeight(0.25)
dense_weight_model.fit(training_labels)

balanced_train_loader = create_dataloader(train_dataset, BATCH_SIZE, dense_weight_model)

loss =  nn.MSELoss()
model = load_resnet_model('resnet50', num_classes=1)
optimizer = optim.Adam(model.parameters(), LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train_model(model, loss, optimizer, scheduler, balanced_train_loader, val_loader, device, ex, epochs=EPOCH)

In [ ]:
ex = 'experiments/ex_10'

dense_weight_model = DenseWeight(0.5)
dense_weight_model.fit(training_labels)

balanced_train_loader = create_dataloader(train_dataset, BATCH_SIZE, dense_weight_model)

loss =  nn.MSELoss()
model = load_resnet_model('resnet50', num_classes=1)
optimizer = optim.Adam(model.parameters(), LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train_model(model, loss, optimizer, scheduler, balanced_train_loader, val_loader, device, ex, epochs=EPOCH)

In [ ]:
ex = 'experiments/ex_11/'

dense_weight_model = DenseWeight(0.75)
dense_weight_model.fit(training_labels)

balanced_train_loader = create_dataloader(train_dataset, BATCH_SIZE, dense_weight_model)

loss =  nn.MSELoss()
model = load_resnet_model()
optimizer = optim.Adam(model.parameters(), LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train_model(model, loss, optimizer, scheduler, balanced_train_loader, val_loader, device, ex, epochs=EPOCH)

In [ ]:
ex = 'experiments/ex_12/'

dense_weight_model = DenseWeight(1)
dense_weight_model.fit(training_labels)

balanced_train_loader = create_dataloader(train_dataset, BATCH_SIZE, dense_weight_model)

loss =  nn.MSELoss()
model = load_resnet_model()
optimizer = optim.Adam(model.parameters(), LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train_model(model, loss, optimizer, scheduler, balanced_train_loader, val_loader, device, ex, epochs=EPOCH)

# Experiment 13: Batched based Monte Carlo Loss

In [ ]:
ex = 'experiments/ex_13'

init_noise_sigma = 1.0
sigma_lr = 0.001
loss = BMCLoss(init_noise_sigma)
model = load_resnet_model('resnet50', num_classes=1)
optimizer = optim.Adam(model.parameters(), lr=LR)
optimizer.add_param_group({'params': loss.noise_sigma, 'lr': sigma_lr, 'name': 'noise_sigma'})
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)

train_model(model, loss, optimizer, scheduler, train_loader, val_loader, device, ex, epochs=50)